In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jax_smi
jax_smi.initialise_tracking()

In [3]:
from scripts.train_phi_sae import main
main()

In [ ]:
from scripts.train_gated_sae import main
main(is_xl=True, layer=20, cache_batch_size=256)
# main()

In [ ]:
from scripts.train_gpt2_sae import main
main(restore="weights/gpt2-20-base.safetensors", layer=32)
# python -m scripts.train_gpt2_sae --save_steps 0 --is_xl=False --layer=9
# main(save_steps=0, is_xl=False, layer=1)